## Data skills block

1. Clean the dataset. Point out anything you find, that required cleaning.
2. Plot a count of bookings over time. Point out anything you notice about the
timeseries.
3. Analyse our activity categories. Tell us anything interesting you find about how
they differ in net revenue (NR) and gross merchandise value (GMV). Are there
other differences between categories that you notice?
4. Some devices have unique booking behaviour. What is unique about device_ids 4 and 5?


Tools used : Python

### 1. Clean the dataset. Point out anything you find, that required cleaning

#### Step 1: Importing required python libraries for whole process

In [1]:
import importlib
libraries = ['plotly', 'matplotlib', 'numpy', 'pandas']
for library in libraries:
    try:
        importlib.import_module(library)
        print(f"{library} is already installed.")
    except ImportError:
        print(f"{library} is not installed.so intitating the installation")
        import pip
        pip.main(['install', library])

print("All required libraries are installed.")


plotly is already installed.
matplotlib is already installed.
numpy is already installed.
pandas is already installed.
All required libraries are installed.


 **NOTE: Run this cell before moving to next cell**

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import matplotlib as plt
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

#### Step 2: Importing the dataset and preliminary data content analysis

In [ ]:
df = pd.read_csv("Fact_orders - fact_orders.csv")
df.columns = df.columns.str.replace(" ","") # to remove the spaces before and after the text in each columns

In [ ]:
df.columns #analysing the columns in the dataframe

In [ ]:
df.head() # I am analysing the first 5 data in the dataframe

In [ ]:
df.info(verbose = True) # Detailed summary of the columns and datatypes

**Observations**: I could observe that the **date_of_checkout** and **date_of_travel** need to be converted in timezone format. Also the **activity_category** and **device_id** columns should be treated as **category** datatype.

In [ ]:
df.describe()

**Observations**: 
-The maximum price of  **Net_Revenue** is 1143.4 euros  and the **GMV** is 5251 euros which is an **outlier** since we could see that the third quartile (Q3) or the value below which 75% of the data 41.3for net_revenue and 163.9 for GMV  The minimum price value of **Net_Revenue** is -19.54 euros and **GMV** is 0.


**Observations:**: In real business context this confition is not possible where GMV and net revenue is 0 so we need to remove this data

In [ ]:
df.isnull().sum()

**Observations:** There are no NA values in columns except in **activity_category** having **6734** null values. I need to analyse the values of net revenue and gross merchandise value of this blank values to make decision whether I need to remove all or keep required rows. 

Based on the analysis, I have decided not to remove all the NA (blank) values in the 'activity_category' column. Instead, for further analysis, I will focus on the net revenue of the blank values in the 'activity_category' column, specifically those that are greater than the **mean value of the 'net_revenue' column**.

In [ ]:
# we are grouping the activity_category based on mean net revenue and unique customer id count
activity_category_stats = df.groupby('activity_category').agg({'net_revenue': 'mean', 'customer_id': 'nunique'}).reset_index()
activity_category_stats = activity_category_stats.sort_values('net_revenue', ascending=False)
print(activity_category_stats)


**Observations**: 
1. Number of Activity Categories: The dataset consists of **11** distinct activity categories, excluding any NA values.

2. Popular Activity Category: Among the available activity categories, **"Attraction Ticket"** seems to be the most popular choice among users, based on the number of bookings.

3. Net Revenue Analysis: The activity category with the highest average net revenue is **"Multi Day Trip"**. However, it is interesting to note that only **288** customers chose this package over a span of three years. The lowest net revenue is in **Transfer** activity category

#### Step 3: Cleaning the data steps and designing one single function

So instead of writing the each operations in each line. I am trying to create **one cleaning function** applicable for this dataset.

**These are the steps undertaken for cleaning the data in the below function:**
- Changing the datatypes
- Change the time format to correct UTC format
- Renaming the columns
- Rounding the numerical values
- Removing the unwanted net revenue and gross merchandise value based on real business context(Normally GMV > Net_Revenue)
- Handling the null values
- Removing outliers for better data quality

In [ ]:

def data_cleaning(df):
    # Changing the datatypes of device_id and activity_category as categorical datatype
    df['device_id'] = df['device_id'].astype('category')
    df['activity_category'] = df['activity_category'].astype('category')

    # Converting the object type of date_of travel and date_of_checkout as date time considering time zone as UTC (Coordinated Universal Time).
    df['date_of_travel'] = pd.to_datetime(df['date_of_travel'], utc=True)
    df['date_of_checkout'] = pd.to_datetime(df['date_of_checkout'], utc=True)

    # Renaming columnsfor more understanding and readability
    df.rename(columns={'date_of_checkout': 'checkout_date', 
                       'date_of_travel': 'travelling_date', 
                       'net_revenue': 'net_revenue(€)', 
                       'gross_merchandise_value': 'gross_merchandise_value(€)'}, inplace=True)

    # Round numeric columns to 2 decimal places in  net_revenue and gross_merchandise_column
    df['net_revenue(€)'] = df['net_revenue(€)'].round(2)
    df['gross_merchandise_value(€)'] = df['gross_merchandise_value(€)'].round(2)

    # in real business context the gross merchandise value usually greater than net_revenue. In some cases only the 
    # net_revenue value and gross merchandise values same if there is no no additonal fees/charges, no commission fees.
    
    df = df[df['gross_merchandise_value(€)'] >= df['net_revenue(€)']]

    # Removing the rows with both values net revenue and  gross merchandise value is zero.
    df = df[(df['net_revenue(€)'] != 0) & (df['gross_merchandise_value(€)'] != 0)]
    
    # Selecting rows where activity_category is not null and if its null values then only taking the net_revenue > mean_net revenue
    mean_net_revenue = df['net_revenue(€)'].mean()
    df = df[(df['activity_category'].notnull()) | ((df['activity_category'].isnull()) & (df['net_revenue(€)'] > mean_net_revenue))]

    # Removing outliers in net revenue column to ensure more data quality 
    Q1 = df['net_revenue(€)'].quantile(0.25)
    Q3 = df['net_revenue(€)'].quantile(0.75)
    IQR = Q3 - Q1
    lower_boundary = Q1 - 1.5 * IQR
    upper_boundary = Q3 + 1.5 * IQR
    df = df[(df['net_revenue(€)'] >= lower_boundary) & (df['net_revenue(€)'] <= upper_boundary)]

    # Removing outliers in gross merchandise value column
    Q1 = df['gross_merchandise_value(€)'].quantile(0.25)
    Q3 = df['gross_merchandise_value(€)'].quantile(0.75)
    IQR = Q3 - Q1
    lower_boundary = Q1 - 1.5 * IQR
    upper_boundary = Q3 + 1.5 * IQR
    df = df[(df['gross_merchandise_value(€)'] >= lower_boundary) & (df['gross_merchandise_value(€)'] <= upper_boundary)]
    return df


In [ ]:
cleaned_df = data_cleaning(df)
cleaned_df

In [ ]:
cleaned_df.isnull().sum()

Here we removed **4972 null values** from the dataset.

In [ ]:
cleaned_df.describe()

#### Step 4: Changing the null values more readable format

In [ ]:
cleaned_df['activity_category'] = cleaned_df['activity_category'].cat.add_categories(['Miscellaneous'])
cleaned_df['activity_category'] = cleaned_df['activity_category'].fillna('Miscellaneous')

Here I am labelling the **NaN values** to **"Miscellaneous"** category for upcoming analysis.

In [ ]:
cleaned_df['activity_category'].unique()

### 2. Plot a count of bookings over time. Point out anything you notice about the timeseries.

In [ ]:
cleaned_df

#### Step 1: Understanding the customer behaviour by analysing the purchasing and travelling data

In [ ]:

cleaned_df['checkout_date_only'] = cleaned_df['checkout_date'].dt.date # extracting the date in checkout_date column

customers_checkout = cleaned_df.groupby('checkout_date_only').size().reset_index(name='count') # grouping the no_of checkouts based on time

cleaned_df['travelling_date_only'] = cleaned_df['travelling_date'].dt.date# extracting the date in travelling_date column

customers_travelled = cleaned_df.groupby('travelling_date_only').size().reset_index(name='count')# grouping the no of travellers travelled based on time

# Creating plots for analysing users who did checkout and users who travelled over year from 2017 to 2019
fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                    subplot_titles=['Count of customers who did Checkouts', 'Count of customers who travelled'])

# first sub-plot for analysing users who did checkout over time
fig.add_trace(go.Scatter(x=customers_checkout['checkout_date_only'], y=customers_checkout['count'],
                         mode='lines', name='Count of users who did checkouts'), row=1, col=1)


# second sub plot for analysing users who took tour packages and travelled
fig.add_trace(go.Scatter(x=customers_travelled['travelling_date_only'], y=customers_travelled['count'],
                         mode='lines', name='Count of travelers'), row=2, col=1)

# main layout design and labelling
fig.update_layout(height=600, width=800, title='Count of travelers over Time')


fig.update_xaxes(title_text='Date', row=2, col=1)

fig.update_yaxes(title_text='Number of Customers', row=1, col=1)
fig.update_yaxes(title_text='Number of Customers', row=2, col=1)

fig.show()



#### Analysis of users who did checkout
- **Peak Checkout Day**: On **April 6, 2018**, the highest number of checkouts was recorded, with a total of **464** users completing    their transactions. This indicates a significant level of activity and potential revenue on that day.

- **Checkout Trend**: The year 2018 is the year most of customers purchased the services comparing 3 years.After **July 27, 2018**, the number of checkouts starts to decline gradually. This suggests that the checkout activity might have decreased over time.It may be due to company issues or marketing campaigns failure.

- **End of Checkouts**: It is interesting to note that there were **no checkouts** recorded after **December 30, 2018**. This could indicate a shift in user behavior or a decline in demand during that period.Also there is net revenue also generated after this period( no checkouts).

- **Seasonal Patterns**: Comparing the checkout patterns over the course of three years, it is apparent that the majority of users tend to make their tour package purchases during the months ranging from April to July and December end. This indicates a seasonal trend where more users are inclined to engage in travel bookings during this time period.

#### Analysis of users who travelled.
- The analysis of user travel patterns over the past three years reveals that July has been the most popular month for travel. In particular, on July 28, 2018, we observed the highest number of users traveling, with approximately 486 users. This indicates a consistent trend of increased travel activity during the month of July. Additionally, there is a noticeable peak in customer travel at the end of December, suggesting that customers often find time for traveling during this period. This might be **summer and christmas vacations** where users travelling alot comparing to other time periods

- In the analysis of checkouts, I identified a significant drop in user activity from December 29, 2018 (432 users) to only 25 users in January 2019. This decline in activity after the holiday season could be attributed to various factors such as the end of vacation periods or reduced travel demand during that particular period.

- Furthermore, when examining the checkout analysis, I observe a consistent trend indicating that the majority of people availed the company's tour packages in the year 2018. This suggests that the tour packages offered by the company were particularly attractive to customers during that year.


#### Step 2: Understanding the customer pre-booking behaviour before their travelling plans

In [ ]:
cleaned_df['time_difference'] = (cleaned_df['travelling_date'] - cleaned_df['checkout_date']).dt.days

bins = [0, 10, 20, 30, np.inf]  
labels = ['0-10 days', '10-20 days', '20-30 days', '30+ days']

cleaned_df['time_range'] = pd.cut(cleaned_df['time_difference'], bins=bins, labels=labels, right=False)


unique_customer_counts = cleaned_df.groupby('time_range')['customer_id'].nunique()
counts_df = pd.DataFrame( {'Unique Customer Count': unique_customer_counts})
counts_df.plot(kind='bar')
plt.xlabel('Time Range')
plt.ylabel('Count of Customers')
plt.title('Customer Prebooking Date Range Before Travel')
plt.show()


In [ ]:
activity_time_revenue = cleaned_df.groupby(['activity_category', 'time_range'])['net_revenue(€)'].mean().reset_index()

activity_time_revenue_table = activity_time_revenue.pivot(index='activity_category', columns='time_range', values='net_revenue(€)')

# Plotting the table as a heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(activity_time_revenue_table, cmap='YlGnBu', annot=True, fmt='.2f', cbar=True)
plt.xlabel('Time Range')
plt.ylabel('Activity Category')
plt.title('Mean Net Revenue by Activity Category and Time Range')
plt.show()


**Observations and Recommendations**: Based on the analysis of the booking data, I have observed the following insights from a business perspective:

1. The majority of customers tend to purchase tour packages and excursions within **0-10** days prior to the actual traveling date. This suggests that customers often make last-minute decisions when planning their trips.

2. To leverage this booking pattern, it would be beneficial to initiate targeted marketing campaigns at least 10 days before the seasonal periods or popular travel dates. By doing so, the tour company can capture the attention of potential customers during the critical decision-making window and increase the likelihood of conversions.Also we need o focus more in the activity category type **"Multi-Day Trip""** and **"Miscellaneous"** (Blank activity category values) need more focus on marketing startgies since they provide more net revenue.

3. By focusing marketing efforts on the recommended timeframe, we can attract a larger customer base and drive more bookings. It is essential to create compelling promotional content, highlight the unique features of our tour packages and excursions, and offer attractive incentives to encourage customers to book early.

4. Additionally, collaborating with local travel agencies, hotels, and popular tourist attractions could provide opportunities for cross-promotion. By establishing partnerships, we can expand our reach and attract customers who are already interested in visiting the destination.

5. It is crucial to identify and prioritize seasonal periods when designing marketing campaigns. Understanding the peak travel seasons, major holidays, and events in different destinations will help us tailor  strategies and allocate resources effectively.

#### Step 3: Analysing user travelling interests over seasons

In [ ]:
cleaned_df['month'] = cleaned_df['travelling_date'].dt.month # extracting month from travelling date

season_mapping = {1: 'Winter', 2: 'Winter', 3: 'Spring', 4: 'Spring', 5: 'Spring', 6: 'Summer',
                  7: 'Summer', 8: 'Summer', 9: 'Fall', 10: 'Fall', 11: 'Fall', 12: 'Winter'} # mapping months to seasons
cleaned_df['season'] = cleaned_df['month'].map(season_mapping)
cleaned_df['season'] = cleaned_df['season'].astype('category') # making the season column to category datatype


In [ ]:
season_customer_count = cleaned_df.groupby('season')['customer_id'].nunique().reset_index(name='count')
total_count = season_customer_count['count'].sum()
fig = px.pie(season_customer_count, values='count', names='season',
             hole=0.5, title='Customers travelling interests by Season')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(annotations=[dict(text=f"Total: {total_count} customers", showarrow=False)])
fig.show()


**Observations:**
Based on my analysis of 90293 users, we have observed the following trends in customer travel preferences.
- During the analysis period, it was found that **38.3%** of the customers preferred to travel during the **summer** season, while **25.2%** preferred the **spring** season.These two seasons accounted for the majority of the customer bookings, indicating a strong preference for traveling during these periods.
- To capitalize on this trend and improve conversion rates, the company should focus on targeting these customer segments with tailored marketing strategies and attractive offers.
- For **existing users**, the tour company can offer exclusive discounts and packages for their next year's travel, aiming to retain their loyalty and increase their conversion rates.It is crucial to design these offers in a way that doesn't significantly impact net revenue, ensuring a balance between customer satisfaction and business profitability.
- To attract **new users**, the company can  explore collaborations with travel influencers, attractive marketing campigns and introducing combo tour packages. By partnering with influential individuals in the travel industry, we can leverage their reach and influence to promote our services and attract new customers.
- Creating enticing offers specifically for new users, in collaboration with these influencers, can help generate interest and encourage them to choose our travel services over competitors.

### 3. Analyse our activity categories. Tell us anything interesting you find about how they differ in net revenue (NR) and gross merchandise value (GMV). Are there other differences between categories that you notice?

#### Step 1: Analysing the average difference of Gross Merchandise Value and Net_Revenue

First I am trying to find the average GMV and average net revenue difference.

In [ ]:

#grouping the average net revenue and average GMV for each activity category
activity_avg = cleaned_df.groupby('activity_category').agg({'net_revenue(€)': 'mean', 'gross_merchandise_value(€)': 'mean'}).reset_index()

# Plotting bar chart to show the extend between the difference of net revenue and GMV
fig = px.bar(activity_avg, x='activity_category', y=['net_revenue(€)', 'gross_merchandise_value(€)'],
             barmode='group', title='Average Mean of Net Revenue and Gross Merchandise Value by Activity Category',
             labels={'value': 'Euros€', 'activity_category': 'Activity Category'})

fig.update_layout(yaxis_title='Average Euros')
fig.show()



In [ ]:


# Calculate the average difference between GMV and net revenue for each activity category
activity_diff_avg = cleaned_df.groupby('activity_category')['gross_merchandise_value(€)', 'net_revenue(€)'].mean()
activity_diff_avg['difference'] = activity_diff_avg['gross_merchandise_value(€)'] - activity_diff_avg['net_revenue(€)']
activity_diff_avg.reset_index(inplace=True)

# Plot the average difference between GMV and net revenue for each activity category
fig = px.bar(activity_diff_avg, x='activity_category', y='difference',
             title='Average Difference between GMV and Net Revenue by Activity Category',
             labels={'difference': 'Difference (€)', 'activity_category': 'Activity Category'})

fig.update_layout(yaxis_title='Average Euros')
fig.show()


**Observations and Recommendations**: we could see that the category Miscellaneous( the category whose activity_category blank values) and Multi day trips have high difference between Gross merchandise value and net revenue.
So it may be due to operational costs and commissions in an online travel tour and excursion booking company.

So the company need to do startegies to reduce the operational costs and commissions.
1.  Review and optimize internal processes to eliminate inefficiencies and reduce operational costs. This could involve automating manual tasks, improving inventory management, and implementing effective resource allocation.
2. Monitoring and control expenses by regularly reviewing expenses, identifying areas of overspending, and taking steps to reduce unnecessary expenditures.
3. Analyze pricing structures and strategies to ensure they are optimized for profitability. Consider factors such as demand, competition, and customer willingness to pay.
4. Implementing analytics tools for data-driven decision-making, and utilizing customer relationship management (CRM) systems for targeted marketing efforts.
5. Focus on targeted marketing campaigns, customer segmentation, and personalized offerings to increase conversion rates and customer loyalty.
6. Review commission structures for third-party bookings and partnerships. Assess the profitability and value provided by each partner and adjust commission rates accordingly. 
7. Implement Agile practices for brainstorming ideas for creating strategies for reducing costs.

#### Step 2: Analysing the negative net_revenue values in active categories

In [ ]:
negative_net_revenue = cleaned_df[cleaned_df['net_revenue(€)'] < 0]['activity_category']
negative_net_revenue

**Observations**: There is only 6 transactions have negative net revenue. this may be due to discounts and low pricing offers provided to customers.

#### Step 3: Analysing the booking frequency per activity category

In [ ]:
# Grouping 'activity_category' and count the number of bookings based on checkout date count
booking_frequency = cleaned_df.groupby('activity_category')['checkout_date'].count().reset_index(name='booking_count')

# Sorting booking count in descending order
booking_frequency = booking_frequency.sort_values('booking_count', ascending=False)

fig = go.Figure(data=go.Bar(y=booking_frequency['activity_category'],
                           x=booking_frequency['booking_count'],
                           orientation='h'))

fig.update_layout(title='Booking Frequency by Activity Category',
                  xaxis_title='Booking Count',
                  yaxis_title='Activity Category')

fig.update_layout(xaxis=dict(tickmode='linear', dtick=2500))

fig.update_layout(height=600, width=800)

fig.show()


**Observations and Recommendations**: 
- After analyzing 3 years of data, it is evident that **Attraction Ticket** and **Day Trip** packages are the most attractive packages for customers. These offerings have potential to generate high revenue and attracted a large number of customers. It is recommended to continue promoting and marketing these packages to further increase customer engagement and revenue. Also optmizing the business costs to incerease high net return value.

- On the other hand, the data shows that **Multi-Day Trip** and **Miscellaneous** activity categories have high operational costs and relatively low customer interest. It is advisable for the company analysts to have discussions with the business heads regarding the viability of these packages. They should evaluate whether it is worthwhile to continue offering these packages or consider alternative options that align better with customer preferences and cost-effectiveness. Also make not that **Multi day trip** is providing highest **mean net revenue**.

- Additionally, it would be beneficial to analyze the demographics and preferences of customers who opt for Attraction Ticket and Day Trip packages. This analysis can provide valuable insights for designing targeted marketing and revenue strategies to attract more customers and maximize profitability.Furthermore, exploring partnerships with local attractions and implementing customer loyalty programs could be effective strategies to enhance customer satisfaction and encourage repeat bookings.

- It is important to regularly monitor and analyze customer feedback and reviews to identify areas for improvement and address any issues promptly. Providing exceptional customer service and personalized experiences can contribute to enhancing customer loyalty and word-of-mouth recommendations.so we need to collect customer feedback for least interested packages.

- Finally, leveraging data analytics and predictive modeling techniques can help in identifying trends, forecasting demand, and optimizing pricing strategies to maximize revenue and profitability. Regularly evaluating and adapting the business strategies based on data-driven insights will contribute to the long-term success of the company.

#### Step 4: Analysing the activity category chose by customers on each seasons

In [ ]:
season_activity_count = cleaned_df.groupby(['season', 'activity_category'])['travelling_date'].count().reset_index(name='count')

pivot_table = season_activity_count.pivot(index='season', columns='activity_category', values='count').fillna(0)

activity_categories = pivot_table.columns.tolist()
seasons = pivot_table.index.tolist()

# setting up different dolors for better readability
colors = ['rgb(31, 119, 180)', 'rgb(255, 127, 14)', 'rgb(44, 160, 44)', 'rgb(214, 39, 40)',
          'rgb(148, 103, 189)', 'rgb(140, 86, 75)', 'rgb(227, 119, 194)', 'rgb(127, 127, 127)',
          'rgb(188, 189, 34)', 'rgb(23, 190, 207)', 'rgb(174, 199, 232)', 'rgb(255, 187, 120)']

# here I am prefering horizontal stacked bar plot for bettere understanding for all audiences
fig = go.Figure()
for i, activity_category in enumerate(activity_categories):
    fig.add_trace(go.Bar(
        y=seasons,
        x=pivot_table[activity_category],
        name=activity_category,
        marker=dict(color=colors[i]),
        orientation='h'
    ))

fig.update_layout(title='Count of Travelers by Season and Activity Category',
                  xaxis_title='Count of travellers travelled',
                  yaxis_title='Season',
                  barmode='stack')

fig.show()


**Observations and Recommendations**: 
- After analyzing the seasonal data of customers who selected activity categories, it is evident that **"Attraction Ticket," "Attraction Walking Tour," "Day Trip,"** and **"City Cruise"** packages have consistently been the most attractive packages in each season. These packages have high potential of high revenue.Highlight their unique features, exclusive offers, and seasonal variations to attract more customers.

- On the other hand, the categories such as **"Miscellaneous," "City Card," and "Transfer"** packages have shown lower customer engagement and travel bookings. These packages have not gained much traction among customers and have contributed less to the overall revenue.The comnpany needs to identify the reasons behind their low customer engagement and take necessary steps to enhance their appeal. This could involve refining package offerings, introducing new features, or exploring collaborations with complementary services or experiences to make them more appealing to customers.
- Continuously improve the overall customer experience across all packages. This includes providing excellent customer service, ensuring smooth and hassle-free bookings, and delivering memorable travel experiences. Customer satisfaction plays a crucial role in generating positive word-of-mouth recommendations, leading to increased bookings and revenue.

### 4.Some devices have unique booking behaviour. What is unique about device_ids 4 and 5?

#### Step 1: Analysing the the customers device usage for checkout

In [ ]:
device_checkout_counts = cleaned_df['device_id'].value_counts()
fig = go.Figure(data=go.Pie(labels=device_checkout_counts.index, values=device_checkout_counts.values))
fig.update_layout(title='Count of Checkout Dates by Device ID')
fig.show()

**Observations and recommendations**: 
Based on the analysis of customer checkouts for tour and excursion packages, the following observations can be made:

- The majority of customers prefer using **device_id 1 and 2** for making their purchases. These device IDs have shown a high level of engagement and activity, indicating that they are popular among customers. It would be advantageous to focus marketing efforts on these devices to further increase revenue and customer engagement.

- **Device_id 4 and 5** have the least usage among customers for checkout in their app or website. This suggests that these devices may not be as popular or user-friendly for the purchasing process. To enhance revenue and expand market reach, it is recommended to explore strategies to improve the user experience on these devices. This could involve optimizing the checkout process, improving the interface, or addressing any technical issues that may be hindering customer engagement.

#### Step 2: Analysing the the customers device usage per revenue

In [ ]:

device_revenue = cleaned_df.groupby('device_id')['net_revenue(€)'].sum().reset_index()

fig = px.bar(device_revenue, x='device_id', y='net_revenue(€)', color='device_id')

fig.update_layout(title='Net Revenue by Device ID',
                  xaxis_title='Device ID',
                  yaxis_title='Net Revenue (€)')

fig.show()


**Observations and recommendations**: 
- **Device 1** and **Device 2**: These two devices have emerged as the **top two revenue generators**. It is evident that customers using these devices are more inclined to make higher-value purchases. By tailoring promotional offers, personalized recommendations, and exclusive deals to attract and retain these high-value customers to increase capitalizing and continue this trends.To maximize revenue, the company should explore **upselling** and **cross-selling** opportunities to customers using Device 1 and Device 2. 
- **Device 4** and **Device 5**:these 2 devices have been identified as the lowest revenue generators. It is crucial to analyze the reasons behind this lower revenue and explore potential strategies to improve performance. 
- By offering bundled packages, exclusive discounts, or joint marketing initiatives, the company can expand its reach, attract new customers, and increase overall revenue in overall for all devices

In [ ]:


correlation_df = cleaned_df.groupby(['device_id', 'activity_category']).size().unstack().fillna(0)

fig = px.bar(correlation_df, orientation='h', barmode='stack')

fig.update_layout(
    title='Correlation between Device ID and Activity Category',
    yaxis_title='Device ID',
    xaxis_title='Count',
    legend_title='Activity Category'
)

fig.show()


#### Step 4: Analysing the the customers device usage on travelling time

In [ ]:


device_counts = cleaned_df.set_index('checkout_date').resample('M')['device_id'].value_counts().unstack()

fig = px.line(device_counts, x=device_counts.index, y=device_counts.columns, 
              labels={'index': 'Time', 'value': 'Count'}, title='Device Usage over Time for Checkout')

fig.update_layout(xaxis=dict(type='date'), yaxis=dict(title='Count of packages purchases'), legend_title='Device ID')

fig.show()


 **Recommendations**: 
 - We need to concentrate on the device_id 1 and device_id 2 for increase customer engagment and conversion by integrating new features like real-time notifications, location-based recommendations, or seamless payment options, to make the booking process more convenient and appealing for users.By providing a superior user experience through these devices, customer satisfaction can be increased, leading to repeat bookings and positive word-of-mouth referrals.
 - Regularly monitor and evaluate the performance and user satisfaction of different devices